In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import foolbox as fb
import tensorflow as tf
import statistics as st
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import load_model

In [3]:
os.chdir(<cats_dogs_dataset>)

In [4]:
import cv2

In [5]:
train_x = np.zeros((8000, 64, 64, 3))

In [6]:
y_train_cats = np.zeros((8000,1))

In [7]:
for i in range(4000):
    img = cv2.imread("cat." + str(i + 1) + ".jpg")
    train_x[(i*2),:,:,:] = cv2.resize(img, (64, 64))
    img = cv2.imread("dog." + str(i + 1) + ".jpg")
    train_x[(i*2 + 1),:,:,:] = cv2.resize(img, (64, 64))

In [8]:
train_x.shape

(8000, 64, 64, 3)

In [9]:
for i in range(4000):
    y_train_cats[(i*2), 0] = 1
    y_train_cats[(i*2 + 1), 0] = 0

In [10]:
m_train = 8000

In [11]:
m_test = 2000

In [12]:
x_train_cats = train_x/255.

In [13]:
x_train_cats.shape

(8000, 64, 64, 3)

In [1]:
# os.getcwd() # for accessing the current directory

In [15]:
os.chdir(<cats_dogs_dataset>) # for changing the current directory to the dataset directory

In [16]:
test_x = np.zeros((2000, 64, 64, 3))

In [17]:
y_test_cats = np.zeros((2000,1))

In [18]:
for i in range(0,1000):
    img = cv2.imread("cat." + str(i + 1 + 4000) + ".jpg")
    test_x[(i*2),:,:,:] = cv2.resize(img, (64, 64))
    img = cv2.imread("dog." + str(i + 1 + 4000) + ".jpg")
    test_x[(i*2 + 1),:,:,:] = cv2.resize(img, (64, 64))

In [19]:
test_x.shape

(2000, 64, 64, 3)

In [20]:
for i in range(1000):
    y_test_cats[(i*2), 0] = 1
    y_test_cats[(i*2 + 1), 0] = 0

In [21]:
x_test_cats = test_x/255.

In [22]:
x_test_cats.shape

(2000, 64, 64, 3)

In [26]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'), 
    tf.keras.layers.Dense(1, activation='sigmoid')  
    ])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 62, 62, 16)        448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 31, 31, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 29, 29, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)             

In [27]:
model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])
history = model.fit(          x = x_train_cats,
                              y = y_train_cats,
                              steps_per_epoch=5,
                              epochs=30,
                              verbose=2)

Epoch 1/30
5/5 - 2s - loss: 0.7622 - accuracy: 0.4985
Epoch 2/30
5/5 - 2s - loss: 0.6887 - accuracy: 0.5705
Epoch 3/30
5/5 - 2s - loss: 0.6787 - accuracy: 0.5788
Epoch 4/30
5/5 - 2s - loss: 0.7248 - accuracy: 0.5666
Epoch 5/30
5/5 - 2s - loss: 0.6681 - accuracy: 0.5985
Epoch 6/30
5/5 - 2s - loss: 0.6626 - accuracy: 0.6122
Epoch 7/30
5/5 - 2s - loss: 0.7173 - accuracy: 0.5804
Epoch 8/30
5/5 - 2s - loss: 0.6465 - accuracy: 0.6540
Epoch 9/30
5/5 - 2s - loss: 0.6317 - accuracy: 0.6425
Epoch 10/30
5/5 - 2s - loss: 0.6748 - accuracy: 0.6034
Epoch 11/30
5/5 - 2s - loss: 0.6441 - accuracy: 0.6215
Epoch 12/30
5/5 - 2s - loss: 0.6237 - accuracy: 0.6553
Epoch 13/30
5/5 - 2s - loss: 0.6676 - accuracy: 0.6125
Epoch 14/30
5/5 - 2s - loss: 0.6118 - accuracy: 0.6809
Epoch 15/30
5/5 - 2s - loss: 0.6371 - accuracy: 0.6310
Epoch 16/30
5/5 - 2s - loss: 0.6381 - accuracy: 0.6260
Epoch 17/30
5/5 - 2s - loss: 0.5920 - accuracy: 0.6976
Epoch 18/30
5/5 - 2s - loss: 0.6338 - accuracy: 0.6476
Epoch 19/30
5/5 - 2

In [28]:
Y = model.predict(x_test_cats)
y_actual = np.argmax(Y, axis = 1)
y_actual = np.expand_dims(y_actual, axis = -1)
acc_non_pruned = (sum(y_actual == y_test_cats)/ m_test) * 100
print("acc_non_pruned = " + str(acc_non_pruned))

acc_non_pruned = [50.]


In [29]:
#preprocessing = dict(flip_axis=-1, mean=[103.939, 116.779, 123.68])
bounds = (0, 1)
fmodel = fb.TensorFlowModel(model, bounds=bounds)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [30]:
fmodel = fmodel.transform_bounds((0, 1))

In [31]:
attack = fb.attacks.FGSM()

In [32]:
epsilon = 0.03

In [33]:
train_x_tf = tf.convert_to_tensor(x_train_cats, np.float32)

In [34]:
train_x_tf.shape

TensorShape([8000, 64, 64, 3])

In [35]:
train_y_sqz = np.squeeze(y_train_cats)

In [36]:
train_y_sqz.shape

(8000,)

In [37]:
train_y_tf = tf.convert_to_tensor(train_y_sqz, np.int32)

In [38]:
train_y_tf.shape

TensorShape([8000])

In [39]:
test_x_tf = tf.convert_to_tensor(x_test_cats, np.float32)

In [40]:
test_x_tf.shape

TensorShape([2000, 64, 64, 3])

In [41]:
test_y_sqz = np.squeeze(y_test_cats)

In [42]:
test_y_sqz.shape

(2000,)

In [43]:
test_y_tf = tf.convert_to_tensor(test_y_sqz, np.int32)

In [44]:
test_y_tf.shape

TensorShape([2000])

In [45]:
import numpy as np
epsilons = np.linspace(0.0, 0.005, num=20)

In [46]:
raw_train, clipped_train, is_adv_train = attack(fmodel, train_x_tf, train_y_tf, epsilons = 0.001)

InvalidArgumentError: Received a label value of 1 which is outside the valid range of [0, 1).  Label values: 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0

In [ ]:
is_adv_train[10:50]

In [ ]:
raw_test, clipped_test, is_adv_test = attack(fmodel, test_x_tf, test_y_tf, epsilons = 0.001)

In [ ]:
is_adv_test[10:50]

In [ ]:
np.sum(is_adv_train == True)

In [ ]:
np.sum(is_adv_test == True)

In [ ]:
fb.distances.l2(train_x_tf[10:50], clipped_train[10:50])

In [ ]:
clipped_train.shape

In [ ]:
clipped_train_m = np.array(clipped_train)

In [ ]:
clipped_train_m.shape

In [ ]:
clipped_test.shape

In [ ]:
clipped_test_m = np.array(clipped_test)

In [ ]:
clipped_test_m.shape

In [ ]:
x_train_cifar_adv = np.zeros((8000, 64, 64, 3))

In [ ]:
x_test_cifar_adv = np.zeros((2000, 64, 64, 3))

In [ ]:
label_train = np.ones((y_train_cifar.shape[0], y_train_cifar.shape[1]))

In [ ]:
label_test = np.ones((y_test_cifar.shape[0], y_test_cifar.shape[1]))

In [ ]:
label_train.shape

In [ ]:
label_test.shape

In [ ]:
for i in range(y_train_cifar.shape[0]):
    if is_adv_train[i] == True:
        label_train[i,0] = 0

In [ ]:
for i in range(y_test_cifar.shape[0]):
    if is_adv_test[i] == True:
        label_test[i,0] = 0

In [ ]:
for i in range(y_train_cifar.shape[0]):
    if is_adv_train[i] == True:
        x_train_cifar_adv[i,:,:,:] = clipped_train_m[i,:,:,:]
    else:
        x_train_cifar_adv[i,:,:,:] = x_train_cifar[i,:,:,:]

In [ ]:
for i in range(y_test_cifar.shape[0]):
    if is_adv_test[i] == True:
        x_test_cifar_adv[i,:,:,:] = clipped_test_m[i,:,:,:]
    else:
        x_test_cifar_adv[i,:,:,:] = x_test_cifar[i,:,:,:]

In [ ]:
# training initialization
mini = 6250
batch = [1, 2, 3, 4, 5, 6, 7, 8]
adversarial_nonpruned = 0
adversarial_total = 0
flipped = np.zeros((12500, 32, 32, 3))
indi = np.zeros((12500,1))
buffer = np.zeros((6250,32,32,3))
buffer_label = np.zeros((6250,1))
buffer_adv = np.zeros((6250,1))

In [ ]:
# training process
for i in range(1, len(batch)):
    #np.random.seed(0)
    if i == 1:
        start = (i-1)*mini
        end = i*mini
        buffer = x_train_cifar_adv[start:end,:,:,:]
        buffer_label = y_train_cifar[start:end,:]
        buffer_adv = label_train[start:end,:]
        
    start = i*mini
    end = (i + 1)*mini
    ip = x_train_cifar_adv[start:end,:,:,:]
    ip_label = y_train_cifar[start:end,:]
    ip_adv = label_train[start:end,:]
    combined = np.concatenate((buffer, ip), axis=0)
    combined_label = np.concatenate((buffer_label, ip_label), axis=0)
    combined_adv = np.concatenate((buffer_adv, ip_adv), axis=0)
    
    for j in range(combined.shape[0]):
        flipped[j,:,:,:] = np.fliplr(combined[j,:,:,:])
    
    if i == 1:
        model_c = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2), 
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'), 
        tf.keras.layers.Dense(10, activation='softmax')  
        ])
        model_c.summary()
        model_c.compile(optimizer=RMSprop(lr=0.001),
              loss='sparse_categorical_crossentropy',
              metrics = ['accuracy'])
        model_f = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2), 
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'), 
        tf.keras.layers.Dense(10, activation='softmax')  
        ])
        model_f.summary()
        model_f.compile(optimizer=RMSprop(lr=0.001),
              loss='sparse_categorical_crossentropy',
              metrics = ['accuracy'])

        history = model_c.fit(x = combined,
                              y = combined_label,
                              steps_per_epoch=5,
                              epochs=15,
                              verbose=2)
        history = model_f.fit(x = flipped,
                              y = combined_label,
                              steps_per_epoch=5,
                              epochs=15,
                              verbose=2)
        model_c.save('contrast_modelc')
        model_c.save_weights('contrast_modelc_weights.h5')
        model_f.save('contrast_modelf')
        model_f.save_weights('contrast_modelf_weights.h5')
    else:
        model_c=load_model('contrast_modelc')
        model_c.load_weights('contrast_modelc_weights.h5')  
        model_f=load_model('contrast_modelf')
        model_f.load_weights('contrast_modelf_weights.h5') 
        history = model_c.fit(x = combined,
                              y = combined_label,
                              steps_per_epoch=5,
                              epochs=15,
                              verbose=2)
        history = model_f.fit(x = flipped,
                              y = combined_label,
                              steps_per_epoch=5,
                              epochs=15,
                              verbose=2)
        model_c.save('contrast_modelc')
        model_c.save_weights('contrast_modelc_weights.h5')
        model_f.save('contrast_modelf')
        model_f.save_weights('contrast_modelf_weights.h5')
  

    from tensorflow.keras import backend as K
    layerIndex = 8
    func1 = K.function([model_c.get_layer(index=0).input], model_c.get_layer(index=layerIndex).output)
    func2 = K.function([model_f.get_layer(index=0).input], model_f.get_layer(index=layerIndex).output)
    layeroutput1 = func1(combined)  
    layeroutput2 = func2(flipped)

    
    z = layeroutput1
    zi = layeroutput2
    
    g = np.multiply(z,zi)
    S = np.max(g, axis = 1)
    s = np.expand_dims(S, axis = -1)
    contrast_score = s
    
    indi = np.zeros(((2*mini),1))
    for j in range(2*mini):
        indi[j,0] = j
    
    n = 2*mini
    for p in range(n-1):
        for q in range(n-1-p):
            if contrast_score[q,0] < contrast_score[q+1,0]:
                contrast_score[q+1,0], contrast_score[q,0] = contrast_score[q,0], contrast_score[q+1,0]
                indi[q+1,0], indi[q,0] = indi[q,0], indi[q+1,0]
    
    index = indi[:mini]
    print(index.shape)
    for l in range(index.shape[0]):
        buffer[l,:,:,:] = combined[int(index[l,0]),:,:,:]
        buffer_label[l,:] = combined_label[int(index[l,0]),:]
        buffer_adv[l,:] = combined_adv[int(index[l,0]),:]
    
    adversarial_nonpruned = mini - sum(buffer_adv)
    adversarial_total = 2*mini - sum(combined_adv)
        
    
    adversarial_pruned = adversarial_total - adversarial_nonpruned
    print("adversarial images pruned = " + str(adversarial_pruned) + " among " + str(adversarial_total) + " images" + "resulting" + str((adversarial_pruned/adversarial_total) * 100) + "% pruning")
    
    
    if i == 1:
        modelP = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(32, 32, 3)),
            tf.keras.layers.MaxPooling2D(2,2),
            tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
            tf.keras.layers.MaxPooling2D(2,2), 
            tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
            tf.keras.layers.MaxPooling2D(2,2),
            tf.keras.layers.Flatten(), 
            tf.keras.layers.Dense(256, activation='relu'), 
            tf.keras.layers.Dense(10, activation='softmax')  
        ])
        modelP.summary()
        modelP.compile(optimizer=RMSprop(lr=0.001),
            loss='sparse_categorical_crossentropy',
            metrics = ['accuracy'])
        
        historyP = modelP.fit(x = buffer,
                              y = buffer_label,
                              steps_per_epoch=5,
                              epochs=15,
                              verbose=2)
        modelP.save('pruned_model')
        modelP.save_weights('pruned_model_weights.h5')
    else:
        modelP=load_model('pruned_model')
        modelP.load_weights('pruned_model_weights.h5')
        historyP = modelP.fit(x = buffer,
                              y = buffer_label,
                              steps_per_epoch=5,
                              epochs=15,
                              verbose=2)
        modelP.save('pruned_model')
        modelP.save_weights('pruned_model_weights.h5')
    YP = modelP.predict(buffer)
    y_actualP = np.argmax(YP, axis = 1)
    y_actualP = np.expand_dims(y_actualP, axis = -1)      
    acc_pruned = (sum(y_actualP == buffer_label)/ mini) * 100
    #accuracy_pruned.append(acc_pruned)
    
    print("acc_pruned = " + str(acc_pruned))
    
    if i == 1:
        model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(32, 32, 3)),
            tf.keras.layers.MaxPooling2D(2,2),
            tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
            tf.keras.layers.MaxPooling2D(2,2), 
            tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
            tf.keras.layers.MaxPooling2D(2,2),
            tf.keras.layers.Flatten(), 
            tf.keras.layers.Dense(256, activation='relu'), 
            tf.keras.layers.Dense(10, activation='softmax')  
        ])
        model.summary()
        model.compile(optimizer=RMSprop(lr=0.001),
              loss='sparse_categorical_crossentropy',
              metrics = ['accuracy'])
        
        history = model.fit(  x = combined[0:end,:,:,:],
                              y = combined_label[0:end,:],
                              steps_per_epoch=5,
                              epochs=15,
                              verbose=2)
        model.save('nonpruned_model')
        model.save_weights('nonpruned_model_weights.h5')
    else:
        model=load_model('nonpruned_model')
        model.load_weights('nonpruned_model_weights.h5')        
        history = model.fit(  x = combined[0:end,:,:,:],
                              y = combined_label[0:end,:],
                              steps_per_epoch=5,
                              epochs=15,
                              verbose=2)
        model.save('nonpruned_model')
        model.save_weights('nonpruned_model_weights.h5')
    Y = model.predict(combined[0:end,:,:,:])
    y_actual = np.argmax(Y, axis = 1)
    y_actual = np.expand_dims(y_actual, axis = -1) 
    acc_non_pruned = (sum(y_actual == combined_label[0:end])/ (2*mini)) * 100
    #accuracy_non_pruned.append(acc_non_pruned)
    print("acc_non_pruned = " + str(acc_non_pruned))


In [ ]:
adversarial_nonpruned_images = mini - sum(buffer_adv)
adversarial_nonpruned_images

In [ ]:
# testing initialization
mini = 1250
batch = [1, 2, 3, 4, 5, 6, 7, 8]
adversarial_nonpruned = 0
adversarial_total = 0
flipped = np.zeros((2500, 32, 32, 3))
indi = np.zeros((2500,1))
buffer = np.zeros((1250,32,32,3))
buffer_label = np.zeros((1250,1))

In [ ]:
# testing process
for i in range(1, len(batch)):
    #np.random.seed(0)
    if i == 1:
        start = (i-1)*mini
        end = i*mini
        buffer = x_test_cifar_adv[start:end,:,:,:]
        buffer_label = y_test_cifar[start:end,:]
        buffer_adv = label_test[start:end,:]
        
    start = i*mini
    end = (i + 1)*mini
    ip = x_test_cifar_adv[start:end,:,:,:]
    ip_label = y_test_cifar[start:end,:]
    ip_adv = label_test[start:end,:]
    combined = np.concatenate((buffer, ip), axis=0)
    combined_label = np.concatenate((buffer_label, ip_label), axis=0)
    combined_adv = np.concatenate((buffer_adv, ip_adv), axis=0)
    for j in range(combined.shape[0]):
        flipped[j,:,:,:] = np.fliplr(combined[j,:,:,:])
    
    layeroutput1 = func1(combined)  
    layeroutput2 = func2(flipped)

    
    z = layeroutput1
    zi = layeroutput2
    
    g = np.multiply(z,zi)
    S = np.max(g, axis = 1)
    s = np.expand_dims(S, axis = -1)
    contrast_score = s
    
    indi = np.zeros((2*mini,1))
    for j in range(2*mini):
        indi[j,0] = j
    
    n = 2*mini
    for p in range(n-1):
        for q in range(n-1-p):
            if contrast_score[q,0] < contrast_score[q+1,0]:
                contrast_score[q+1,0], contrast_score[q,0] = contrast_score[q,0], contrast_score[q+1,0]
                indi[q+1,0], indi[q,0] = indi[q,0], indi[q+1,0]
    
    index = indi[:mini]
    print(index.shape)
    for l in range(index.shape[0]):
        buffer[l,:,:,:] = combined[int(index[l,0]),:,:,:]
        buffer_label[l,:] = combined_label[int(index[l,0]),:]
        buffer_adv[l,:] = combined_adv[int(index[l,0]),:]
    
    adversarial_nonpruned = 1250 - sum(buffer_adv)
    adversarial_total = 2500 - sum(combined_adv)
        
    
    adversarial_pruned = adversarial_total - adversarial_nonpruned
    print("adversarial images pruned = " + str(adversarial_pruned) + " among " + str(adversarial_total) + " images" + "resulting" + str((adversarial_pruned/adversarial_total) * 100) + "% pruning")
    
    
    
    
    YP = modelP.predict(buffer)
    y_actualP = np.argmax(YP, axis = 1)
    y_actualP = np.expand_dims(y_actualP, axis = -1)      
    acc_pruned = (sum(y_actualP == buffer_label)/ mini) * 100
    print("acc_pruned = " + str(acc_pruned))
    
    
    Y = model.predict(combined)
    y_actual = np.argmax(Y, axis = 1)
    y_actual = np.expand_dims(y_actual, axis = -1) 
    acc_non_pruned = (sum(y_actual == combined_label)/ (2*mini)) * 100
    print("acc_non_pruned = " + str(acc_non_pruned))